# Import Libraries

In [1]:
import jax
import jax.numpy as jnp
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 데이터 준비
- Iris 데이터셋을 사용하여 분류 문제 해결

In [2]:
# 데이터 로드 및 전처리
iris = load_iris()
X = iris.data
y = iris.target

# 원-핫 인코딩
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# JAX 배열로 변환
X_train = jnp.array(X_train)
X_test = jnp.array(X_test)
y_train = jnp.array(y_train)
y_test = jnp.array(y_test)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!


Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



W0000 00:00:1732282714.723858  963293 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1732282714.737170  963293 service.cc:145] XLA service 0x158314e20 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732282714.737181  963293 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1732282714.738540  963293 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1732282714.738554  963293 mps_client.cc:384] XLA backend will use up to 11452858368 bytes on device 0 for SimpleAllocator.


# 모델 정의
- 신경망의 파라미터와 예측 함수를 정의

In [3]:
# 가중치 초기화 함수
def init_params(key, input_dim, hidden_dim, output_dim):
    key1, key2, key3 = jax.random.split(key, 3)
    W1 = jax.random.normal(key1, (input_dim, hidden_dim))
    b1 = jnp.zeros(hidden_dim)
    W2 = jax.random.normal(key2, (hidden_dim, output_dim))
    b2 = jnp.zeros(output_dim)
    return W1, b1, W2, b2

# 신경망 예측 함수
def predict(params, x):
    W1, b1, W2, b2 = params
    hidden = jax.nn.relu(jnp.dot(x, W1) + b1)
    logits = jnp.dot(hidden, W2) + b2
    return logits

# 손실 함수 및 그래디언트 계산
- 손실 함수(크로스 엔트로피)와 그래디언트를 정의

In [4]:
# 크로스 엔트로피 손실
def loss_fn(params, x, y):
    logits = predict(params, x)
    probs = jax.nn.softmax(logits)
    return -jnp.mean(jnp.sum(y * jnp.log(probs), axis=1))

# 그래디언트 계산
grad_fn = jax.grad(loss_fn)

# 학습 루프
- JAX로 학습 루프 구현

In [5]:
# 하이퍼파라미터
learning_rate = 0.01
epochs = 100
hidden_dim = 16

# 모델 초기화
key = jax.random.PRNGKey(42)
params = init_params(key, input_dim=4, hidden_dim=hidden_dim, output_dim=3)

# 학습 루프
for epoch in range(epochs):
    grads = grad_fn(params, X_train, y_train)  # 그래디언트 계산
    params = [(p - learning_rate * g) for p, g in zip(params, grads)]  # 파라미터 업데이트

    if epoch % 10 == 0:
        loss = loss_fn(params, X_train, y_train)
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 18.9897
Epoch 10, Loss: 1.8351
Epoch 20, Loss: 1.6515
Epoch 30, Loss: 1.5376
Epoch 40, Loss: 1.4198
Epoch 50, Loss: 1.3060
Epoch 60, Loss: 1.1891
Epoch 70, Loss: 1.0735
Epoch 80, Loss: 0.9633
Epoch 90, Loss: 0.8597


# 모델 평가
- 테스트 데이터에서 성능 평가

In [6]:
# 예측 함수
def accuracy(params, x, y):
    logits = predict(params, x)
    preds = jnp.argmax(logits, axis=1)
    labels = jnp.argmax(y, axis=1)
    return jnp.mean(preds == labels)

acc = accuracy(params, X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

Test Accuracy: 0.7000
